In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ambig\\jupiter_notebook\\Projects\\premium-price-prediction\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\ambig\\jupiter_notebook\\Projects\\premium-price-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir : Path
    input_path: Path
    output_path : Path
    raw_data_path: Path
    test_data_path : Path
    train_data_path : Path
    
    
    

In [5]:
from src.Premium_Price_Prediction.constants import *
from src.Premium_Price_Prediction.utils.common import read_yaml , create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            input_path=config.input_path,
            output_path=config.output_path,
            raw_data_path=config.raw_data_path,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path  # Fixed colon to equal sign
        )
        return data_ingestion_config


In [ ]:
import os
import  urllib.request as request
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from src.Premium_Price_Prediction import logger  # Ensure the logger is correctly imported

class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.input_path = config.input_path
        self.output_path = config.output_path  # Output path where files will be saved
        self.raw_data_path = config.raw_data_path  # Path for raw data
        self.train_data_path = config.train_data_path  # Path for training data
        self.test_data_path = config.test_data_path  # Path for testing data
        
    def load_data(self):
        """Loads data from an Excel file."""
        try:
            df = pd.read_excel(self.input_path)
            logger.info("Data loaded successfully.")
            print(df.head())  # Display the first few rows of the DataFrame
            return df  # Return the loaded DataFrame for further use
        except Exception as e:
            logger.error(f"Error occurred while loading data: {str(e)}")
            raise e
    
    def save_data(self) -> None:
        """Loads data and saves it in data ingestion artifacts.
        
        Raises:
            Exception: For any errors encountered while saving the data.
        """
        try:
            df = self.load_data()  # Load data
            
            # Ensure directory exists for output
            os.makedirs(os.path.dirname(self.output_path), exist_ok=True)  
            
            os.makedirs(os.path.dirname(self.output_path), exist_ok=True)  
            # Save the raw data
            df.to_csv(self.raw_data_path, index=False)
            logger.info(f"Raw data saved successfully in {self.raw_data_path}")

          
        except Exception as e:
            logger.error(f"Error occurred while saving data: {str(e)}")
            raise e


In [9]:
try : 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.load_data()
    data_ingestion.save_data()

except Exception as e:
    logger.error(f"Error occurred during data ingestion: {str(e)}")
    raise e

[2024-11-04 11:52:26,688: INFO: common: 30] YAML file : config\config.yaml loaded successfully
[2024-11-04 11:52:26,716: INFO: common: 30] YAML file : params.yaml loaded successfully
[2024-11-04 11:52:26,716: INFO: common: 30] YAML file : schema.yaml loaded successfully
[2024-11-04 11:52:26,722: INFO: common: 50] Directory artifacts created successfully.
[2024-11-04 11:52:26,722: INFO: common: 50] Directory artifacts/data_ingestion created successfully.
[2024-11-04 11:52:42,664: INFO: 1438235562: 20] Data loaded successfully.
   Age  Gender     Region Marital_status  Number Of Dependants BMI_Category  \
0   26    Male  Northwest      Unmarried                     0       Normal   
1   29  Female  Southeast        Married                     2      Obesity   
2   49  Female  Northeast        Married                     2       Normal   
3   30  Female  Southeast        Married                     3       Normal   
4   18    Male  Northeast      Unmarried                     0   Overweig